<font size="5"> Data Manipulation using DASK and Causal Inference using Diff-in-Diff approach

In [1]:
import requests, json, time, statistics, aiohttp, pandas as pd, numpy as np, altair as alt

from datetime import datetime, date, timedelta
from dateutil.relativedelta import relativedelta

# DASK
import dask.array as da
import dask.dataframe as dd
import dask.bag as bag
from dask.distributed import LocalCluster, Client

pd.set_option('display.max_columns', 500)


<font size="5"> Find Causal effect of Coronavirus on home prices in Los Angeles County

<font size="3"> Intuition:
    
> We hypothesize home prices to fall due to Covid. We think Covid may have led to weakened demand for houses, 
due to some people losing jobs hence moving to rental units, some people falling sick hence pausing their plans of buying new house (or upgrading to a bigger house), changes in attitudes towards perceived risk of real estate investing among other factors.  

<font size="3"> What is diff-in-diff (DiD) method?
    
> DiD method is a common statistical technique used to draw causal inferences from observational data. It calculates the causal effect of a treatment (Examples: sudden change like a new tax introduction or a Govt policy change or Covid hitting world, etc) on an outcome variable (e.g. median home price of single family homes) by comparing the average change over time in the outcome variable for the treatment group, compared to the average change over time for the control group.

> With the unexpected advent of Covid-19 in our county, it is reasonable to ask how it affected home prices. This is difficult to find out because in order to truly know how those home prices in our county were causally impacted, we need to consider how those home prices would be had the county never experienced Covid-19 (the counterfactual). However, the Covid-19 did hit the county and we never get to observe how those home prices would have fared without Covid-19. 
    
> DiD uses the outcome of the control group as a proxy / counterfactual for what would have occurred in the treatment group had there been no treatment. The difference in the average post-treatment outcomes between the treatment and control groups is a measure of the causal effect.

Datasets:
> The datasets we use to explore this question can be found here https://api.developer.attomdata.com/docs and https://usafacts.org/visualizations/coronavirus-covid-19-spread-map/state/california
    

In [2]:
s = pd.read_csv('/Users/adityahpatel/Desktop/PYTHON PROGRAMS/Milestone Project/covid_confirmed_usafacts.csv')
cali = s[s.State == 'CA']   # Pulling out all counties of California, same can be done for NY state
cali.head(5)

,countyFIPS,County Name,State,StateFIPS,2020-01-22,2020-01-23,2020-01-24,2020-01-25,2020-01-26,2020-01-27,2020-01-28,2020-01-29,2020-01-30,2020-01-31,2020-02-01,2020-02-02,2020-02-03,2020-02-04,2020-02-05,2020-02-06,2020-02-07,2020-02-08,2020-02-09,2020-02-10,2020-02-11,2020-02-12,2020-02-13,2020-02-14,2020-02-15,2020-02-16,2020-02-17,2020-02-18,2020-02-19,2020-02-20,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25,2020-02-26,2020-02-27,2020-02-28,2020-02-29,2020-03-01,2020-03-02,2020-03-03,2020-03-04,2020-03-05,2020-03-06,2020-03-07,2020-03-08,2020-03-09,2020-03-10,2020-03-11,2020-03-12,2020-03-13,2020-03-14,2020-03-15,2020-03-16,2020-03-17,2020-03-18,2020-03-19,2020-03-20,2020-03-21,2020-03-22,2020-03-23,2020-03-24,2020-03-25,2020-03-26,2020-03-27,2020-03-28,2020-03-29,2020-03-30,2020-03-31,2020-04-01,2020-04-02,2020-04-03,2020-04-04,2020-04-05,2020-04-06,2020-04-07,2020-04-08,2020-04-09,2020-04-10,2020-04-11,2020-04-12,2020-04-13,2020-04-14,2020-04-15,2020-04-16,2020-04-17,2020-04-18,2020-04-19,2020-04-20,2020-04-21,2020-04-22,2020-04-23,2020-04-24,2020-04-25,2020-04-26,2020-04-27,2020-04-28,2020-04-29,2020-04-30,2020-05-01,2020-05-02,2020-05-03,2020-05-04,2020-05-05,2020-05-06,2020-05-07,2020-05-08,2020-05-09,2020-05-10,2020-05-11,2020-05-12,2020-05-13,2020-05-14,2020-05-15,2020-05-16,2020-05-17,2020-05-18,2020-05-19,2020-05-20,2020-05-21,2020-05-22,2020-05-23,2020-05-24,2020-05-25,2020-05-26,2020-05-27,2020-05-28,2020-05-29,2020-05-30,2020-05-31,2020-06-01,2020-06-02,2020-06-03,2020-06-04,2020-06-05,2020-06-06,2020-06-07,2020-06-08,2020-06-09,2020-06-10,2020-06-11,2020-06-12,2020-06-13,2020-06-14,2020-06-15,2020-06-16,2020-06-17,2020-06-18,2020-06-19,2020-06-20,2020-06-21,2020-06-22,2020-06-23,2020-06-24,2020-06-25,2020-06-26,2020-06-27,2020-06-28,2020-06-29,2020-06-30,2020-07-01,2020-07-02,2020-07-03,2020-07-04,2020-07-05,2020-07-06,2020-07-07,2020-07-08,2020-07-09,2020-07-10,2020-07-11,2020-07-12,2020-07-13,2020-07-14,2020-07-15,2020-07-16,2020-07-17,2020-07-18,2020-07-19,2020-07-20,2020-07-21,2020-07-22,2020-07-23,2020-07-24,2020-07-25,2020-07-26,2020-07-27,2020-07-28,2020-07-29,2020-07-30,2020-07-31,2020-08-01,2020-08-02,2020-08-03,2020-08-04,2020-08-05,2020-08-06,2020-08-07,2020-08-08,2020-08-09,2020-08-10,2020-08-11,2020-08-12,2020-08-13,2020-08-14,2020-08-15,2020-08-16,2020-08-17,2020-08-18,2020-08-19,2020-08-20,2020-08-21,2020-08-22,2020-08-23,2020-08-24,2020-08-25,2020-08-26,2020-08-27,2020-08-28,2020-08-29,2020-08-30,2020-08-31,2020-09-01,2020-09-02,2020-09-03,2020-09-04,2020-09-05,2020-09-06,2020-09-07,2020-09-08,2020-09-09,2020-09-10,2020-09-11,2020-09-12,2020-09-13,2020-09-14,2020-09-15,2020-09-16,2020-09-17,2020-09-18,2020-09-19,2020-09-20,2020-09-21,2020-09-22,2020-09-23,2020-09-24,2020-09-25,2020-09-26,2020-09-27,2020-09-28,2020-09-29,2020-09-30,2020-10-01,2020-10-02,2020-10-03,2020-10-04,2020-10-05,2020-10-06,2020-10-07,2020-10-08,2020-10-09,2020-10-10,2020-10-11,2020-10-12,2020-10-13,2020-10-14,2020-10-15,2020-10-16,2020-10-17,2020-10-18,2020-10-19,2020-10-20,2020-10-21,2020-10-22,2020-10-23,2020-10-24,2020-10-25,2020-10-26,2020-10-27,2020-10-28,2020-10-29,2020-10-30,2020-10-31,2020-11-01,2020-11-02,2020-11-03,2020-11-04,2020-11-05,2020-11-06,2020-11-07,2020-11-08,2020-11-09,2020-11-10,2020-11-11,2020-11-12,2020-11-13,2020-11-14,2020-11-15,2020-11-16,2020-11-17,2020-11-18,2020-11-19,2020-11-20,2020-11-21,2020-11-22,2020-11-23,2020-11-24,2020-11-25,2020-11-26,2020-11-27,2020-11-28,2020-11-29,2020-11-30,2020-12-01,2020-12-02,2020-12-03,2020-12-04,2020-12-05,2020-12-06,2020-12-07,2020-12-08,2020-12-09,2020-12-10,2020-12-11,2020-12-12,2020-12-13,2020-12-14,2020-12-15,2020-12-16,2020-12-17,2020-12-18,2020-12-19,2020-12-20,2020-12-21,2020-12-22,2020-12-23,2020-12-24,2020-12-25,2020-12-26,2020-12-27,2020-12-28,2020-12-29,2020-12-30,2020-12-31,2021-01-01,2021-01-02,2021-01-03,2021-01-04,2021-01-05,2021-01-06,2021-01-07,2021-01-08,2021-01-09,2021-01-10,2021-01-11,2021-01-12,2021-01-13,2021-01-14,2021-01-15,

In [3]:
# (*) Identify 2 counties (treatment and control) preferably in same state 
#     where Covid cases reached a threshold (A2: 10K cases, when ppl startes losing jobs and not when
#     1st case was reported...companies realized this virus is here to stay) at 2 different points of time..

# (*) Los Angeles county hit 10,000 cases on 04/14/2020
#     San Francisco county hit 10,000 cases on 09/08/2020
#     If we define Treatment as a county hitting 10,000 cases, it means from 04/14/2020 to 09/08/2020, 
#     LA was in treatment period, but SF was not. 
#     Therefore, we pick LA as treatment group, SF as control group for DiD. 
#     To simplfy analyses, we pick a 1 month period from 04/14/2020 to 05/14/20 as post-treatment period, AND
#     04/14/2019 to 05/14/2019 as pre-treatment period. This allows us to make a YOY comparision at same points of
#     time 

# DEFINITIONS:
#  Outcome variable: Median Home Price of Single Family Residence (SFR) in US dollars. 


# Parallel Trends assumption(Untestable in practice) for DiD: We assume that if Covid had not hit, the time trend in Los Angeles would 
# match the time trend of San Francisco. This seems reasonable to us considering both are top counties in SAME state 
# and have many similarities. There is nothing unique to LA or SF county that changed over time. Anything that 
# confounder changes over time affects treatment (LA) and control (SF) group in the same way. i.e. there are no time-varying 
# LA or SF specific confounders i.e. anything that changes over time that might affect median home prices cannot systematically
# differ between LA and SF (treatment and control)

# In other words, we There are no systematic differences in the changes of unobserved confounders.

# Calculate simple Difference-in-Difference of Median Home Price of Single Family Homes as the causal effect of
# Covid on Home Prices in Los Angeles county

<font size="3"> 

In [4]:
# this function reads a batch of JSON files and returns a DASK bag containing all transactions. Each item in the 
# Dask bag is a dictionary

def work_read(filepath):
    b = bag.read_text(filepath).map(json.loads) 
    b = b.flatten()   # without flattening the bag, the results in the bag will be useless due to excessive nesting
    return b

In [5]:
# this filter function takes as input a dictionary containing data on a single transaction and returns 
# transaction if it contains data on Single Family Homes(SFR) in pre-treatment period (2019/4/14 to 2019/5/15). 

def filters_pretreatment(bag_item):  # each bag_item will be a dictionary
    condition1 = 'sale' in bag_item.keys()
    condition2 = 'proptype' in bag_item['summary'].keys()
    condition3 = 'amount' in bag_item['sale'].keys()
    condition4 = 'saleamt' in bag_item['sale']['amount'].keys()
    date = datetime.strptime(bag_item['sale']['saleTransDate'], '%Y-%m-%d') 
    condition5 = (date >= datetime(2019,4,14)) & (date < datetime(2019,5,15))
    condition6 = 'saleTransDate' in bag_item['sale'].keys()
    condition7 = bag_item['summary']['proptype'] == 'SFR'
    return (condition1 and condition2) and (condition3 and condition4) and (condition5 and condition6)and condition7

In [6]:
# this filter function takes as input a dictionary containing data on a single transaction and returns 
# transaction if it contains data on Single Family Homes (SFR)in post-treatment period (2020/4/14 to 2020/5/15). 

def filters_posttreatment(bag_item):  
    condition1 = 'sale' in bag_item.keys()
    condition2 = 'proptype' in bag_item['summary'].keys()
    condition3 = 'amount' in bag_item['sale'].keys()
    condition4 = 'saleamt' in bag_item['sale']['amount'].keys()
    date = datetime.strptime(bag_item['sale']['saleTransDate'], '%Y-%m-%d') 
    condition5 = (date >= datetime(2020,4,14)) & (date < datetime(2020,5,15))
    condition6 = 'saleTransDate' in bag_item['sale'].keys()
    condition7 = bag_item['summary']['proptype'] == 'SFR'
    return (condition1 and condition2) and (condition3 and condition4) and (condition5 and condition6)and condition7


In [7]:
# This function takes as input a dictionary and returns the column of interest(sales price) 
def extractor(record):
    return record['sale']['amount']['saleamt'] 

<font size="4"> Read batch files obtained by Attom API calls for both counties into Dask Bags 

In [8]:
path_losangeles = '/Users/adityahpatel/Desktop/PYTHON PROGRAMS/Housing Full Dataset/CO06037 *' 
path_sanfrancisco = '/Users/adityahpatel/Desktop/PYTHON PROGRAMS/Housing Full Dataset/CO06075 *'

bag_losangeles = work_read(path_losangeles)  # bag contains all transations in LA county from 1/1/2019 
bag_sanfrancisco = work_read(path_sanfrancisco)

In [9]:
# bag_la.count().compute()  Optional: before filtering there are 150501 dictionaries in bag
# bag_la.take(1)   Optional: this is like .head(1) but for bag instead of pandas df

<font size="4"> Data Processing using DASK 

In [10]:
# Preprocessing Required for Dask. Needed to make sure there are no keyerrors while parsing dictionaries in Bag

bag_losangeles = bag_losangeles.filter(lambda x:'saleTransDate' in x['sale'].keys()).filter(lambda x:'proptype' in x['summary'].keys())
bag_sanfrancisco = bag_sanfrancisco.filter(lambda x:'saleTransDate' in x['sale'].keys()).filter(lambda x:'proptype' in x['summary'].keys())

In [11]:
# Dask Bag filtering. Retains only those transactions in pre-treatment period for SingleFamilyResidences 

bag_losangeles_pre = bag_losangeles.filter(filters_pretreatment)  
bag_sanfrancisco_pre = bag_sanfrancisco.filter(filters_pretreatment)  

# Dask Bag filtering. Retains only those transactions in pre-treatment period for SingleFamilyResidences 

bag_losangeles_post = bag_losangeles.filter(filters_posttreatment)  
bag_sanfrancisco_post = bag_sanfrancisco.filter(filters_posttreatment) 

# Put above 4 Dask bags into a python list

bags = [bag_losangeles_pre, bag_sanfrancisco_pre, bag_losangeles_post, bag_sanfrancisco_post]

# Apply Dask Mapper function 

def median_generator(bag):
    bag = bag.map(extractor)   # Dask mapper used to extract sales price column
    L = list(bag)
    median = statistics.median(L)
    return median

median_la_pre, median_sf_pre,median_la_post,median_sf_post = [median_generator(bag) for bag in bags]    

In [12]:
print('Median home price of SingleFamilyHome in Los Angeles in pre-treatment period = {}'.format(median_la_pre))
print('Median home price of SingleFamilyHome in SanFrancisco in pre-treatment period = {}'.format(median_sf_pre))

print('Median home price of SingleFamilyHome in Los Angeles in post-treatment period = {}'.format(median_la_post))
print('Median home price of SingleFamilyHome in SanFrancisco in post-treatment period = {}'.format(median_sf_post))

Median home price of SingleFamilyHome in Los Angeles in pre-treatment period = 667250.0
Median home price of SingleFamilyHome in SanFrancisco in pre-treatment period = 1632500.0
Median home price of SingleFamilyHome in Los Angeles in post-treatment period = 665000.0
Median home price of SingleFamilyHome in SanFrancisco in post-treatment period = 1500000


In [13]:
causaleffect_percentage = ((665000-667250)/665000) - ((1500000 - 1632500)/1500000)
print ("The causal effect of Coronavirus on SFR home prices in Los Angeles = {} %)".format(causaleffect_percentage * 100))


The causal effect of Coronavirus on SFR home prices in Los Angeles = 8.49498746867168 %)


<font size="4"> Finding: *Covid-19 actually caused single family home prices in Los Angeles County to increase!*

<font size="3"> We would be interested in extending this causal analysis to more complex versions of *DiD* (like *"DiD Regression with Fixed Effects" method*) "to include multiple periods (instead of single month) and multiple counties (instead of just 1 treatment county and 1 control county). 
    
<font size="3">The "*DiD regression with Fixed Effects*" method would allow us to report standard error/robustness of causal inference. We could also combine *DiD* method with *Nearest Neighbor Matching* method. This would involve 'matching' known 'treatment' units with simulated counterfactual 'control' units: characteristically equivalent units which did not receive treatment. 
    
<font size="3">*However, it is our intent to restrict the scope of causal inference to "basic" Exploratory Data Analysis as per Milestone # 1 guidelines.*

In [15]:
# End